# cphonon function
### Authors
Written in matlab by Anders Blom, *1999*
Improved by Björn Samuelsson, *2000*
Converted to python Andris Potrebko *2019*

# Here I am just playing around with widgets

In [1]:
# x=5
# slider = widgets.IntSlider()
# slider.value = x
# def on_change(v):
#    global x
#    T = v['new'] 
# slider.observe(on_change, names='value')
# display(slider)

In [2]:
%matplotlib inline
%matplotlib notebook

from scipy.sparse import diags
import numpy as np
from scipy.linalg import eig
from matplotlib import pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive

from IPython.display import display #Java - not there anymore for running cells from widgits

# For plots being displaied right under the shell
#import scipy as sp

# pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

###Plotly Not sure if works
# from plotly.offline import init_notebook_mode, iplot
# from plotly.graph_objs import *

In [3]:
# %matplotlib notebook
# %matplotlib notebook
# from matplotlib import pyplot as plt
# x = np.linspace(0, 2 * np.pi)
# fig = plt.figure()
# ax = fig.add_subplot(1, 1, 1)
# line, = ax.plot(x, np.sin(x))

# def update(w = 1.0):
#     line.set_ydata(np.sin(w * x))
#     fig.canvas.draw_idle()

# interact(update);

# Real code starts here

### Some initial values

In [4]:
N = 10
M1 = 0.5
M2 = 1
C = 1
imp_enabled = 0
Nimp = 5
Mimp = 5
bc = 0

Create force matrix, with 2 on the diagonal and -1 on each side


In [5]:
def CreateForceMatrix(N):
    tmp1 = [-1]*(N-1)
    tmp2 = [2]*N
    diagonals = [tmp1, tmp2, tmp1]  # the diagonals of the force matrix
    A = diags(diagonals, [-1, 0, 1]).todense()
    return(A)
A = CreateForceMatrix(N)

In [6]:
#A

Update the force matrix considering all boundary conditions

In [7]:
def ForceMatrixBoundaryCond(A, bc, N):
    if bc == 2:  # open ends
        A[0, 0] = 1
        A[N-1, N-1] = 1
    elif bc == 1:  # fixed ends
        A[0, 0] = - 2  # this will generate negative eigenvalues, to be removed below
        A[N-1, N-1] = - 2
        A[1, 0] = 0
        A[0, 1] = 0
        A[N - 2, N-1] = 0
        A[N-1, N - 2] = 0
    else:  # periodic boundary cond.
        A[0, N-1] = - 1
        A[N-1, 0] = - 1
    return(A)
A = ForceMatrixBoundaryCond(A, bc, N)

In [8]:
#A

Create mass matrix, take care of impurities

In [9]:
def CreateMassMatrix(N, M1, M2, Nimp, Mimp, imp_enabled):
    Nimp = Nimp-1  # Python counts from 0
    B = np.zeros(shape=(N, N))  # diags([1]*N,0).todense()
    for i in range(N):
        if i % 2 == 0:  # if even
            B[i, i] = 1 / M1
        else:
            B[i, i] = 1 / M2

    if imp_enabled:
        B[Nimp, Nimp] = 1 / Mimp
    return(B)
B = CreateMassMatrix(N, M1, M2, Nimp, Mimp, imp_enabled)

In [10]:
#B

### Solve the eigenvalue equation
We get *omega* - eigenfrequecies and V - the vector of all displacements *u*

Set number of modes for fixed b.c. this number will be reduced by 2 as we remove negative eigenvalues (Two of the atoms don't move)

In [11]:
def EigenEq(A, B, C, bc, N):
    A1 = np.dot(C*B, A)
    D, V = eig(A1)
    D = np.real(D)
    Nmodes = N
    # Find negative eigenvalues and remove corresponding eigenvectors/-values for fixed b.c
    if bc == 1:
        for k in range(2):
            neg = -1
            for i in range(Nmodes):
                if ((D[i] < -1e-06) & (neg == -1)):
                    neg = i
            D = np.concatenate([D[:(neg)], D[(neg+1):]])
            V = np.concatenate([V[:, :(neg)], V[:, (neg+1):]], axis=1)
            Nmodes = Nmodes - 1

    # Abs because D can be slightly negative ex. -6.10622664e-16
    omega = np.sqrt(np.abs(D))
    # sort the eigenvalues and eigenmodes according to the eigenvalues
    ind = np.argsort(omega)
    omega = np.sort(omega)
    V = V[:, ind]

    return(omega, V, Nmodes)
omega, V, Nmodes = EigenEq(A, B, C, bc, N)

In [12]:
# 
# omega
# V
# Nmodes

In [13]:
# take some care if eigenvalue is first or last in vector
# =============================================================================
#             if neg == 1:
#                 D=D[1:]
#                 V=V[1:,:]
#             else:
#                 if neg == Nmodes:
#                     D=D[:(Nmodes - 1)]
#                     V=V[:(Nmodes-1),:]
#                 else:
#                     if neg != 0:
# =============================================================================

### Discrete fourier transform
We use Fast fourier transform algorithm (*fft* function) to implement the discrete fourier transform.
* For the case of *fixed ends* boundary conditions the smallest wave possible can be a wave with the wavelength *2*\(N-1)* - the wave can be antisymetric with the wave outside. This is why we are looking for *2*\(N-1)* wave number, but taking at the end only the first *Nmodes* possible ones.
* For the case of *open ends* - the real periodicity is *2*\N*
* For periodic boundary conditions, the periodicity is *N* but we cut *np.floor(N/2)* <- **?????** 
Ech frequency except zero is doubly degenerate with +K and -K solutions (moving in opposite directions)

Don't get imag of coefficients //Andris **Need to check the open ends if it is symetric**


In [14]:
def FourierTransform(bc, V, N, Nmodes):
    if bc == 2:  # open ended
        wavemax = Nmodes
        Kk = np.fft.fft(V, 2*N, axis=0)
        Kk = Kk[:int(wavemax) + 1, :Nmodes]
        Ksq = np.real(Kk*np.conj(Kk))
        Karg = np.angle(Kk)
    elif bc == 1:  # fixed imaginary part turns out to work better. Still don't get why
        wavemax = Nmodes
        Kk = np.imag(np.fft.fft(V, 2*(N - 1), axis=0))
        Kk = Kk[:wavemax + 1, :Nmodes]
        Ksq = Kk*Kk
        Karg = 'NA'
        # Karg=Na
    else:  # periodic
        wavemax = np.floor(N / 2)
        Kk = np.fft.fft(V, N, axis=0)
        Kk = Kk[:int(wavemax) + 1, :Nmodes]
        Ksq = np.real(Kk*np.conj(Kk))  # can be a bit negative
        Karg = np.angle(Kk)

    Ka = np.argmax(Ksq, axis=0)
    # mx=np.max(Ksq,axis=0)#this one we don't need
    return(Ka, Karg)  # maybe Kk,
Ka, Karg = FourierTransform(bc, V, N, Nmodes)

In [15]:
# Ka
# Karg

In [16]:
# Ksq1=np.imag(Kk)*np.imag(Kk)
# Ksq2=np.real(Kk*np.conj(Kk))

# plt.plot(Ksq1[:,0])
# plt.plot(Ksq2[:,0])

# #plt.plot(np.imag(Ksq1[:,1]))
#plt.plot(np.imag(Ksq2[:,1]))
#plt.plot(np.real(Ksq1[:,1]))
#plt.plot(np.real(Ksq2[:,1]))

In [17]:
# %%capture 
# fig, ax = plt.subplots();
# #wavemax=np.floor(N / 2)
# Ksq=Kk*Kk
# ax.plot(np.real(Ksq[:,1]))
# ax.plot(np.real(Ksq[:,2]))
# #ax.plot(np.real(V[:,1]))
# #ax.plot(np.real(V[:,2]))
# fig

In [18]:
# plt.plot(Ksq[:,])

###Bunch of boring but important manipulations with the results.
* For periodic boundary conditions we have doubly degenerate levels. If we have degenerate levels: give that one with the smallest phase angle a plus sign and the other a minus sign.
* *Ka* values from the fft are only positive. Here we correct the signs filling up the dispersion plot
* Give the corect amplitude for *Ka*

In [19]:
# set Ka to the index of the biggest squared coefficient
def CorrectOmega(Ka, Karg, Nmodes, V, omega, bc, M1, M2, N):
    if not(Nmodes):  # if Nmodes = 0 (case when bc=1 and N=2) gives an error
        maxi = 0
    else:
        maxi = np.argmax(Ka)

    if bc == 0:
        for j in range(Nmodes-1):
            if np.abs(omega[j] - omega[j + 1]) < 1e-06:  # if both omegas "almost"" equal
                v1 = Karg[Ka[j], j]
                v2 = Karg[Ka[j+1], j + 1]
                diff = v2 - v1
                if diff < - np.pi:
                    diff = diff + 2*np.pi
                elif diff > np.pi:
                    diff = diff - 2*np.pi
                # those where maxi+1 is even Ka will be -Ka, see below in this sec.
                if diff > 0 or not((maxi+1) % 2 < 0):
                    V[:, j] = - V[:, j]
                if np.abs(np.abs(diff) - np.pi / 2) > 1e-06:
                    # make them both orthonormal (supposes normated vectors)
                    V[:, j + 1] = V[:, j + 1] - V[:, j] * \
                        np.dot(V[:, j + 1], V[:, j])
                    V[:, j + 1] = V[:, j + 1] / \
                        np.sqrt(np.dot(V[:, j + 1], V[:, j + 1]))

    # Change sigh of every second Ka, depending on which is the maximum Ka
    Ka[(maxi) % 2::2] = -Ka[(maxi) % 2::2]

    # set correct magnitude of Ka
    if bc == 1:
        Ka = Ka*np.pi / (N - 1)
    else:
        Ka = Ka*2*np.pi / N

    # The high Ka values () belong to
    if M1 != M2:
        Ka = Ka*2

        for i in range(Nmodes):
            if np.abs(Ka[i]) > np.pi:
                Ka[i] = np.sign(Ka[i])*(np.abs(Ka[i]) - 2*np.pi)

    # correct sign if the last Ka is on the right boundary to the left side (we define our interval of Ka: [-pi/a, pi/a) )
    for i in range(Nmodes):
        if np.abs(Ka[i] - np.pi) < 1e-06:
            Ka[i] = - Ka[i]
    return(V, omega, Ka)
V, omega, Ka = CorrectOmega(Ka, Karg, Nmodes, V, omega, bc, M1, M2, N)

In [20]:
#V
#omega
#Ka

In [21]:
# Vdiff=V[:(N-1),:]-V[1:N,:]
# if bc==0:
#     Vdiff=np.vstack((Vdiff[:(N-1),:],V[(N-1),:] - V[0,:]))

# Vdiff=V[:(N-1),:]-V[1:N,:]
# if bc==0:
#     Vdiff = np.vstack((Vdiff[:(N-1),:],V[(N-1),:] - V[0,:]))
# Vdiff=np.diag(np.dot(np.transpose(Vdiff),Vdiff)).copy()

# if Nmodes!=0:   #Otherwise give error, because no such element
#     if Vdiff[0] < 1e-06:
#         Vdiff[0]=1
# Vdiff

### set amplitude proportional to classical amplitude of one excitation

In [22]:
def CorrectAmplitude(V, omega, C, N):
    Vdiff = V[:(N-1), :]-V[1:N, :]
    if bc == 0:
        Vdiff = np.vstack((Vdiff[:(N-1), :], V[(N-1), :] - V[0, :]))

    Vdiff = V[:(N-1), :]-V[1:N, :]
    if bc == 0: #Maybe if bc!=1, to incude open ends
        Vdiff = np.vstack((Vdiff[:(N-1), :], V[(N-1), :] - V[0, :]))
    Vdiff = np.diag(np.dot(np.transpose(Vdiff), Vdiff)).copy()

    if Nmodes != 0:  # Otherwise it gives error, because no such element
        if Vdiff[0] < 1e-06:
            Vdiff[0] = 1

    Ch = 4*np.sqrt(2*omega / (C*Vdiff))  
    V = np.dot(V, diags(Ch, 0).todense())
    return(V)


V = CorrectAmplitude(V, omega, C, N)

In [23]:
#V

In [24]:
# Vdiff = V[:(N-1), :]-V[1:N, :]
# if bc == 0:
#     Vdiff = np.vstack((Vdiff[:(N-1), :], V[(N-1), :] - V[0, :]))

# Vdiff = V[:(N-1), :]-V[1:N, :]
# if bc == 0: #Maybe if bc!=1, to incude open ends
#     Vdiff = np.vstack((Vdiff[:(N-1), :], V[(N-1), :] - V[0, :]))
# Vdiff = np.diag(np.dot(np.transpose(Vdiff), Vdiff)).copy()

# if Nmodes != 0:  # Otherwise it gives error, because no such element
#     if Vdiff[0] < 1e-06:
#         Vdiff[0] = 1
# Ch = 4*np.sqrt(2*omega / (C*Vdiff))

### Analytic solution of the dispersion relation

In [25]:
# %%capture
# capture - so that the plot does not get displayed
# plt.ioff()


# fig
# plt.close()

The guy bellow (function) plots dispersion plot from both data and the theoretical calculations

In [26]:
def PlotDisp(C, M1, M2, omega, Ka, ax1):
    #fig, ax = plt.subplots();
    #global ax1
    ax1.cla()
    ax1.plot(Ka, omega, 'bo', label=['simulation'])
    ka = np.linspace(-np.pi, np.pi, 100)
    if M1 == M2:
        analytic = np.sqrt(4*C/M1)*np.abs(np.sin(ka/2))
        ax1.plot(ka, analytic, label=['analytic'])
    else:
        MM = (M1+M2)/(M1*M2)
        analytic1 = np.sqrt(
            C*MM * (1 + np.sqrt(1-2/MM/(M1+M2)*(1-np.cos(ka)))))
        analytic2 = np.sqrt(
            C*MM * (1 - np.sqrt(1-2/MM/(M1+M2)*(1-np.cos(ka)))))
        ax1.plot(ka, analytic1, label=['analytic acustic'])
        ax1.plot(ka, analytic2, label=['analytic optical'])
    ax1.legend(loc='lower left')
    # fig
    # if M1=M2:
#    ax.plot(xa,analytic1,label=['analytic'])
# else:
#PlotDisp(C,M1,M2, omega, Ka)
# fig

In [27]:
#fig, ax = plt.subplots(1,2,1);

This guy plots dispacements at one specific given eigenmode

In [28]:
def PlotEigenmode(V, ModeNr, M1, M2, ax2, imp_enabled, Mimp, Nimp):
    #fig, ax = plt.subplots();
    Nmodes=len(V[:, 1])
    ax2.cla()
    if M1 > M2:
        mark1 = 11
        mark2 = 6
    elif M1 == M2:
        mark1 = mark2 = 6
    else:
        mark1 = 6
        mark2 = 11
    marktype = 'bo' if M1 == M2 else 'go'
    oddatoms = range(1, Nmodes+1, 2)
    evenatoms = range(2, Nmodes+1, 2)
    allatoms = range(1, Nmodes+1)
    ax2.plot(oddatoms, V[::2, ModeNr], 'bo', markersize=mark1)
    ax2.plot(evenatoms, V[1::2, ModeNr], marktype, markersize=mark2)
    ax2.plot(allatoms, V[:, ModeNr], '-y')
    if imp_enabled == 1:
        ax2.plot(Nimp, V[Nimp-1, ModeNr], 'wo', markersize=11)
        ax2.plot(Nimp, V[Nimp-1, ModeNr], 'ro', markersize=np.log(Mimp/M1+1.7)*7)

Finnaly we will make the *__animation__*! We prepare function for the animation to be run later in animation.FuncAnimation later

In [29]:
# %gui qt
# import pyqtgraph as pg
# #from PyQt4 import QtGui
# from pyqtgraph.Qt import QtCore, QtGui
# import numpy as np
# def AnimationGT():
#     win = pg.GraphicsWindow()
#     win.setWindowTitle('pyqtgraph example: Scrolling Plots')


#     # 1) Simplest approach -- update data in the array such that plot appears to scroll
#     #    In these examples, the array size is fixed.
#     p1 = win.addPlot()
#     p2 = win.addPlot()
#     x=np.linspace(0, 6 * np.pi)
#     data1=np.sin(x)
#     #data1=np.random.normal(1,1,10)
#     curve1 = p1.plot(data1)
#     curve2 = p2.plot(data1)
#     ptr1 = 0
#     def update1():
#         global data1, curve1, ptr1
#         data1[:-1] = data1[1:]  # shift data in the array one sample left
#                                 # (see also: np.roll)
#         data1[-1] = data1[0]# np.random.normal() 
#         curve1.setData(data1)

#         ptr1 += 1
#         curve2.setData(data1)
#         curve2.setPos(ptr1, 0)

#     # def update():
#     #     update1()
#     #     update2()
#     #     update3()
#     timer = pg.QtCore.QTimer()
#     timer.timeout.connect(update1)
#     timer.start(50)
# AnimationGT()

In [30]:
#%matplotlib notebook
#import matplotlib.animation
#def AnimationCreator():


### Here we begin the user interface (UI) part using *widgets* :)

In [32]:
# %%capture
# from matplotlib import pyplot as plt
# %matplotlib notebook
# # calling it a second time may prevent some graphics errors
# %matplotlib notebook
# fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2)
# fig.set_size_inches(11.5, 4.5)
# fig2, ax3 = plt.subplots(nrows=1,ncols=1)
# fig2.set_size_inches(10.5, 1.1)


# def update(*rgs):
#     #global MyInteraction.children.step
#     if (M1!=M2)&(bc==0)&(N%2==1):
#         #MyInteraction.children[0].step=2
#         #N=N+1
#         print("It happened")
#     else:
#     #    MyInteraction.children[0].step=1
#         #Nreal.value=N
#         print("Did not")
#     A = CreateForceMatrix(N)
#     A = ForceMatrixBoundaryCond(A, bc, N)
#     B = CreateMassMatrix(N, M1, M2, Nimp, Mimp, imp_enabled)
#     omega, V, Nmodes = EigenEq(A, B, C, bc, N)
#     Ka, Karg = FourierTransform(bc, V, N, Nmodes)
#     V, omega, Ka = CorrectOmega(Ka, Karg, Nmodes, V, omega, bc, M1, M2, N)
#     V = CorrectAmplitude(V, omega, C, N)
#     # print(omega)
#     # print(Ka)
#     # ax.plot(Ka,omega,'bo',label=['simulation']);
#     # fig
#     PlotDisp(C, M1, M2, omega, Ka, ax1)
#     PlotEigenmode(V, ModeNr-1, M1, M2, ax2, imp_enabled, Mimp, Nimp)
    

# MyInteraction=interactive(update, N=(2, 50, 1), C=(0.2, 10, 0.2), bc=[('periodic', 0),('fixed ends', 1), ('open ends', 2)],
#                           M1=(0.1, 5, 0.1), M2=(0.1, 5, 0.1), imp_enabled=[('impurity disabled', 0), ('impurity enabled', 1)],
#                           Nimp=(1, 10, 1), Mimp=(0.1, 5, 0.1),ModeNr=(1, 5, 1))
# #Nreal=widgets.IntSlider(value=10,min=0, max=50, step=1)
# #l=widgets.jslink((MyInteraction.children[0],"value"),(Nreal,"value"))
# NInter=widgets.IntSlider(
#      value=10, description="number of atoms", min=2, max=500, step=1)
# CInter=widgets.IntSlider(
#      value=10, description="C ($\gamma$ in the book)", min=2, max=500)
# bcInter=widgets.IntSlider(
#      value=10, description="boundary condition", min=2, max=500)
# M1Inter=widgets.IntSlider(
#      value=10, description="number of atoms", min=2, max=500)
# M2Inter=widgets.IntSlider(
#      value=10, description="number of atoms", min=2, max=500)
# imp_enabledInter=widgets.IntSlider(
#      value=10, description="number of atoms", min=2, max=500)
# NimpInter=widgets.IntSlider(
#      value=10, description="number of atoms", min=2, max=500)
# MimpInter=widgets.IntSlider(
#      value=10, description="number of atoms", min=2, max=500)
# ModeNrInter=widgets.IntSlider(
#      value=10, description="Mode Number", min=2, max=500)

# NInter.observe(update, "value")


Import some missing libraries and define a function which will calculate $\omega$ and $K_a$ (do everything above) every time some of the main input widgets will be changed. Function *interactive* also defines these widgets.

In [33]:
from ipywidgets import Label, HBox, Layout

Before the definition of the function we also define plots which will be called later. For some buggy reason, the *matplotlib* has to be imported and "*%matplotlib notebook*" has to be called again

In [ ]:
%%capture
from matplotlib import pyplot as plt
%matplotlib notebook
# calling it a second time may prevent some graphics errors
%matplotlib notebook
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2)
fig.set_size_inches(11.5, 4.5)
fig2, ax3 = plt.subplots(nrows=1,ncols=1)
fig2.set_size_inches(10.5, 1.1)

In [139]:
def update(N=10, C=1, bc=0, M1=1, M2=1, imp_enabled=0, Nimp=3, Mimp=1, ModeNr=0):
    #global MyInteraction.children.step
    #if (M1!=M2)&(bc==0)&(N%2==1):
        #MyInteraction.children[0].step=2
        #N=N+1
    #    print("It happened")
    #else:
    #    MyInteraction.children[0].step=1
        #Nreal.value=N
    #    print("Did not")
    A = CreateForceMatrix(N)
    A = ForceMatrixBoundaryCond(A, bc, N)
    B = CreateMassMatrix(N, M1, M2, Nimp, Mimp, imp_enabled)
    omega, V, Nmodes = EigenEq(A, B, C, bc, N)
    Ka, Karg = FourierTransform(bc, V, N, Nmodes)
    V, omega, Ka = CorrectOmega(Ka, Karg, Nmodes, V, omega, bc, M1, M2, N)
    V = CorrectAmplitude(V, omega, C, N)
    # ax.plot(Ka,omega,'bo',label=['simulation']);
    # fig
    PlotDisp(C, M1, M2, omega, Ka, ax1)
    PlotEigenmode(V, ModeNr-1, M1, M2, ax2, imp_enabled, Mimp, Nimp)
    return Ka[ModeNr-1], omega[ModeNr-1]
#
MyInteraction=interactive(update, N=(2, 200, 1), C=(0.2, 10, 0.2), bc=[('periodic', 0),('fixed ends', 1), ('open ends', 2)],
                          M1=(0.1, 5, 0.1), M2=(0.1, 5, 0.1), imp_enabled=[('impurity disabled', 0), ('impurity enabled', 1)],
                          Nimp=(1, 10, 1), Mimp=(0.1, 5, 0.1),ModeNr=(1, 5, 1))
#Nreal=widgets.IntSlider(value=10,min=0, max=50, step=1)
#l=widgets.jslink((MyInteraction.children[0],"value"),(Nreal,"value"))

I agree that the bellow is not the prettiest way of doing it but I am not really a  programmer and just learning about widgets. :D So, first we get acces to all widgets in MyInteraction so that it would be more starightforward doing it. Also, delete names (*.description*) of each widget because otherwise it will appear when we print it using *HBox* later. The output looks nicer if the name is defined at *HBox*.

In [157]:
for widg in MyInteraction.children[:-1]:
    widg.description=""
    widg.continuous_update=False
    #widg.style=style
NInter=MyInteraction.children[0]
CInter=MyInteraction.children[1]
bcInter=MyInteraction.children[2]
M1Inter=MyInteraction.children[3]
M2Inter=MyInteraction.children[4]
imp_enabledInter=MyInteraction.children[5]
NimpInter=MyInteraction.children[6]
MimpInter=MyInteraction.children[7]
ModeNrInter=MyInteraction.children[8]

#Change sizes of the two boxes otherwise it is too large
bcInter.layout=Layout(width='80px')
imp_enabledInter.layout=Layout(width='130px')

Here define a widget that will print out the chosen value of $K_a$ and corresponding $\omega$

In [156]:
### Define printing omega and Ka values
OmegaPrint="Frequency \( \omega\) is <b>{}</b> a.u"
Kprint= "<br> Wave vector \( K_a \) is <b>{}</b> a.u"         
PrintValue=widgets.HTMLMath(
    value='',
    #value=OmegaPrint.format(np.round(MyInteraction.result[1],3))+Kprint.format(np.round(MyInteraction.result[0],2)),
    placeholder='Some HTML',
    #description='Some HTML'
)
def updateHTML(*args):
    PrintValue.value=OmegaPrint.format(np.round(MyInteraction.result[1],3))+Kprint.format(np.round(MyInteraction.result[0],2))

for widg in MyInteraction.children:
    widg.observe(updateHTML, 'value')
    


#### Defining additional widgets and functions
First we define a bottom that will make the two masses equal when we press it

In [115]:
MassEqualBtn = widgets.Button(description='Equalize Masses')
def equalize_Masses(btn_object):
    M2Inter.value = M1Inter.value
MassEqualBtn.on_click(equalize_Masses)

Take care that when we have fixed ends, the mode number is N-2
Afterwards, when we have different basses and periodic boundary conditions, N must be even

In [113]:
def updateMaxValues(*args):
    if bcInter.value==1:
        ModeNrInter.max=NInter.value-2
    else:
        ModeNrInter.max=NInter.value
    NimpInter.max=NInter.value
    
NInter.observe(updateMaxValues, 'value')
bcInter.observe(updateMaxValues, 'value')

def updateNstep(*args):
    if (M1Inter.value!=M2Inter.value)&(bcInter.value==0):
        NInter.step=2
        NInter.value = NInter.value+1 if NInter.value%2 else NInter.value
    else:
        NInter.step=1

M1Inter.observe(updateNstep, 'value')
M2Inter.observe(updateNstep, 'value')
bcInter.observe(updateNstep, 'value')
    
#l=widgets.jslink((MyInteraction.children[0],"value"),(Nreal,"value"))
play = widgets.Play(
     interval=10,
    value=50,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
def MakeAnim(change):
    global x
    global ax3
    w=change["new"]
    ax3.cla()
    ax3.plot(x,w*np.sin(w * x*0.005))

x = np.linspace(0, 2 * np.pi)
ax3.plot(x, np.sin(x))
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
play.observe(MakeAnim, 'value')

Link(source=(Play(value=50, description='Press play', interval=10), 'value'), target=(IntSlider(value=0), 'val…

Define all of the outputs

In [91]:
FirstBox=widgets.HBox([Label('N of atoms'),NInter,Label('C ($\gamma$ in the book)'),CInter,Label('boundary cond.'), bcInter])
Player=widgets.HBox([play, slider])
Masses=widgets.HBox([Label('Mass 1 $M_1$'),M1Inter,Label('Mass 2, $M_2$'), M2Inter, MassEqualBtn])
ImpurityBox=widgets.HBox([imp_enabledInter,Label('Mass of imp.'),MimpInter,Label('Atom nr. of imp.'), NimpInter])
ModeNrInterBox=widgets.HBox([Label('Mode number'),ModeNrInter, PrintValue])
Impurity = widgets.Accordion(children=[Masses,ImpurityBox])
Impurity.set_title(0,'Masses') 
Impurity.set_title(1, 'Impurity')
#PrintBox=HBox(PrintValue)
# fig2, axAnim = plt.subplots();
# l, = axAnim.plot([0,2*np.pi],[-1,1])
# animate = lambda i: l.set_data(t[:i], x[:i])
# ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(t))
#fig
# display(MyInteraction)
# #fig2.show()
# fig2
# display(Player)
#widgets.Box([PrintValue])
#fig.show()

In [38]:
# fig, ax3=plt.subplots(1,1)
# play = widgets.Play(
#      interval=10,
#     value=50,
#     min=0,
#     max=100,
#     step=1,
#     description="Press play",
#     disabled=False,continuous_update=True
# )

# def MakeAnim(change):
#     global x
#     global ax3
#     w=change["new"]
#     ax3.cla()
#     ax3.plot(x,w*np.sin(w * x*0.005))

# x = np.linspace(0, 2 * np.pi)
# ax3.plot(x, np.sin(x))

# slider = widgets.IntSlider(continuous_update=True)
# widgets.jslink((play, 'value'), (slider, 'value'))
# AnimationFtion()
# play.observe(MakeAnim, 'value')
# widgets.HBox([play, slider])

In [138]:
#display(MyInteraction.children[-1])

In [40]:
# def update(N=10,M1=1, M2=1):
#     if (M1!=M2) & (N%2==1):
#         Nreal.value=N+1
#         print("N value automatically changed")

# MyInteraction=interactive(update, N=(2, 50, 1), M1=(0.1, 5, 0.1), M2=(0.1, 5, 0.1), Mimp=(0.1, 5, 0.1))
# Nreal=widgets.IntSlider(value=3,min=0, max=50, step=1,)
# l=widgets.link((MyInteraction.children[0],"value"),(Nreal,"value"))
# display(play)
# display(MyInteraction)

In [41]:
# A=CreateForceMatrix(N)
# A = ForceMatrixBoundaryCond(A, bc, N)
# B = CreateMassMatrix(N, M1, M2, Nimp, Mimp)
# omega, V, Nmodes = EigenEq(A, B, C, bc, N)
# Ka, Karg = FourierTransform(bc,V, N, Nmodes)
# V,omega, Ka=CorrectOmega(Ka, Karg, Nmodes, V, omega, bc, M1, M2, N)
# A,V=CorrectAmplitude(V,omega,C,N)
# fig, ax = plt.subplots();
# ax.plot(Ka,omega,'bo',label=['simulation']);
# #fig
# #Analytic(C,M1,M2)
# #fig
# #plt.plot(V[:,])
# #plt.plot(omega)
# #plt.plot(Ka)
# print(bc)

In [44]:

#ModeNrInter.max